# Freezing mode to create low symmetry structure
## Prepared by Kishwar-E Hasin
(Please suggest corrections to improve the script)

Here, for example, I froze 5 different irreps on high symmetry structure to make a low symmetry structure. You can freeze single or multiple irreps at the same time.

Necessary input files:

1. High symmetry poscar (HS) file
2. Excel file with irrep matrices with given format (need to prepare this file according to the structure)

Calculation Steps:
1. Create matrix with direct position of high symmetry POSCAR file
2. Make lattice vector 
3. Convert high symmetry position matrix in Angstrom : multiplying high symmetry by lattice vector
4. Create sym mode matrices and save in an excel file with the given format.
5. Multiply the matrix by the cell_correction (e.g. array (sqrt 2, sqrt 2, 1)) if z element present with x or y
6. Normalize the irrep matrix
7. Create the distortion matrix,  distortion = Q_element . normalized irrep matrix
8. Create low symmetry matrix (in Angstrom): low symmetry matrix (in Angstrom)A + distortion
9. Create low symmetry direct position matrix : dividing low symmetry matrix (in Angstrom) by lattice vector
10. Write the low symmetry POSCAR on a file


In [23]:
import pandas as pd
import numpy as np
import math
from itertools import islice


high_symmetry_file_name = "HS"
matrix_file_name = "irrep_matrices_regular_RP_20_atom_cell.xlsx"
cell_correction = np.array([np.sqrt(2.), np.sqrt(2.), 1])   # change according to your computational unit cell

#============above 3 lines needed to be changed according to choice======

with open(high_symmetry_file_name) as lines:          
        lattice_parameter = np.genfromtxt(islice(lines, 2, 5))                ## Extract lattice parameter     
        lattice_vector = np.diag(lattice_parameter)
        for i, line in enumerate(lines):                                      ## Count the total number of atoms
            if i == 1:  
                n_atom = sum(map(int, line.strip().split()))      
                break
        high_symmetry = np.genfromtxt(islice(lines, 1, 1 + n_atom))           ## Extract positions from high symmetry POSCAR    
        high_symmetry_angstrom = high_symmetry * lattice_vector               ## High symmetry positions in Angstrom

### Define function to freeze mode of specified amplitude        
def freeze_mode(sheet_name, element_values, mode_name, Q):   
    irrep_data = pd.read_excel(matrix_file_name, sheet_name, skiprows=1)      
    
    displacement = []
    for element_name, value in element_values.items():
        df_element = pd.DataFrame(irrep_data[0:n_atom], columns=[f"x_{element_name}", f"y_{element_name}", f"z_{element_name}"])        
        z_nonzero = np.any(df_element[f"z_{element_name}"] != 0)
        xy_nonzero = np.any((df_element[f"x_{element_name}"] != 0) | (df_element[f"y_{element_name}"] != 0))   
        e_element_matrix = np.array(df_element) 
        if z_nonzero and xy_nonzero:
            e_element_matrix = np.array(df_element) * cell_correction
        
        e_element = e_element_matrix / (np.linalg.norm(e_element_matrix))
        displacement.append(value * e_element)

    sum_displacement = np.sum(displacement, axis=0)
    distortion = Q * sum_displacement
    return distortion


#======== From this to the next barrier line:  you need to change according to your choice===========

#Define contribution/values for each element in a dictionary for different modes 

element_values_m2 = {
    "O" : 0.7253322265235937
    # Define values for mode m2  
}

element_values_m3 = {
    "O" : 0.0009693687057284184
    # Define values for mode m3  
}

element_values_r4 = {
    "A" : -0.07083928604310244,
    "O" : 0.03131198800138041
    # Define values for mode m5
}

element_values_r5 = {
    "O" : 1.1341721859781582
    # Define values for mode m5
}

element_values_x5 = {
    "A" : 0.3183644307430506,
    "O" : -0.17958462947040466
    # Define values for mode m5
}


# Define Q (amplitude factors) for modes. 
Q_m2 = 1 
Q_m3 = 1
Q_r4 = 1
Q_r5 = 1
Q_x5 = 1


# List of mode information tuples: (sheeet name, element value, mode name, amplitude factor Q)
modes = [
    ('M2+', element_values_m2, "m2", Q_m2),
    ('M3+', element_values_m3, "m3", Q_m3),
    ('R4-', element_values_r4, "r4", Q_r4),
    ('R5-', element_values_r5, "r5", Q_r5),
    ('X5-', element_values_x5, "x5", Q_x5)
]

#================= Till this line ====================================================================

# Initialize total distortions to a zero array with the same shape as the first distortion matrix
total_distortions = np.zeros_like(freeze_mode(*modes[0]))

# Calculate and sum distortions
for sheet_name, element_values, mode_name, Q in modes:
    distortion = freeze_mode(sheet_name, element_values, mode_name, Q)
    #print(f"Distortion for {mode_name}:", distortion)
    total_distortions += distortion
#print("Total distortions:", total_distortions) 

low_symmetry_angstrom = high_symmetry_angstrom + total_distortions  # Calculate lsa by adding total_distortions to hsa
low_symmetry_positions = low_symmetry_angstrom / lattice_vector     # Divide lsa by np.diag(lp) to calculate ls

### Write the low symmetry POSCAR in a file ####
with open(high_symmetry_file_name) as f:
    data_list = f.readlines()
    data_list[0] = "new_poscar\n"
    del data_list[8:]
    mat = np.matrix(low_symmetry_positions)
    with open('POSCAR', 'w') as f:
        f.writelines(data_list)
        np.savetxt(f, mat, fmt='  %.16f', delimiter='      ')